In [ ]:
#!/usr/bin/env python
# coding: utf-8

In[10]:

In [ ]:
get_ipython().magic('pylab inline')

Smooth Parameter Variation<br>
=====================================<br>
<br>
This tutorial shows how to smoothly vary simulation parameters as a function of time. <br>
<br>
It is specifically aimed at a predetermined variation, that is a temporal variation defined<br>
by a function that does not depend on the system's state.<br>
<br>
On the other hand, if what you want to achieve is a state dependent variation, then the<br>
better, more efficient, approach is to define a new Model of local dynamics where the <br>
"parameter" you wish to vary is recast as a state-variable.

Setup and Simulator<br>
--------------------<br>
<br>
As usual, we'll geet started by loading some stuff through "lab".

In[11]:

In [ ]:
from tvb.simulator.lab import *
LOG = get_logger('demo')

And then, we simply construct and configure a Region level Simulator:

In[12]:

nitialise a Model, Coupling, and Connectivity.

In [ ]:
oscillator = models.Generic2dOscillator()
white_matter = connectivity.Connectivity(load_default=True)
white_matter.speed = numpy.array([4.0])
white_matter_coupling = coupling.Linear(a=0.0154)

nitialise an Integrator

In [ ]:
heunint = integrators.HeunDeterministic(dt=2**-6)

nitialise some Monitors with period in physical time

In [ ]:
mon_raw = monitors.Raw()
mon_tav = monitors.TemporalAverage(period=2**-2)

undle them

In [ ]:
what_to_watch = (mon_raw, mon_tav)

nitialise a Simulator -- Model, Connectivity, Integrator, and Monitors.

In [ ]:
sim = simulator.Simulator(model = oscillator, 
                          connectivity = white_matter,
                          coupling = white_matter_coupling, 
                          integrator = heunint, 
                          monitors = what_to_watch)
sim.configure()

Temporal Parameter Variation<br>
-----------------------------<br>
<br>
To define the parameter's time course we'll need to specify upfront how long <br>
a block of simulation we're going to run.

In[13]:

In [ ]:
sim_len = 2.0**7

We can use TVB's Equations datatype to create a Gaussian that peaks half <br>
way through our simulation. 

In[14]:

Define a variation of a model parameter as a function of time using an Equation datatype

In [ ]:
t =  numpy.arange(0, sim_len, sim.integrator.dt)

In [ ]:
eqn_t = equations.Gaussian()
eqn_t.parameters["amp"] = 4.2
eqn_t.parameters["midpoint"] = sim_len / 2.0
eqn_t.pattern = t
a = eqn_t.pattern

ake it a variation relative to the default value

In [ ]:
a = sim.model.a + a

Grab the length of what we just created, as we'll need it below

In [ ]:
par_len = a.shape[0]

Let's take a quick look at the parameter variation we just defined, to make sure it looks like what we want...

In[15]:

lot the stimulus

In [ ]:
plot(t, a)

Simulate<br>
--------<br>
<br>
Initialise some empty lists that will contain results of the simulation

In[16]:

In [ ]:
raw_data, raw_time = [], []
tavg_data, tavg_time = [], []

We then run the simulation, much as we normally would, just with an explicit extra<br>
step of updating our Model parameter at each step of the integration.<br>
<br>
**Note:** The use of sim.current_step and the modulo (%) operator here is just to <br>
    make it easier for us to rerun this block later, see below.

In[17]:

un the simulation

In [ ]:
step = sim.current_step
sim.model.a = a[step%par_len]

In [ ]:
for raw, tavg in sim(simulation_length = sim_len):
    
    if not raw is None:
        raw_time.append(raw[0])
        raw_data.append(raw[1])
    
    if not tavg is None:
        tavg_time.append(tavg[0])
        tavg_data.append(tavg[1])
        
    step += 1
    # Change a model parameter at each integration step
    sim.model.a = a[step%par_len]

Plots<br>
-----<br>
<br>
Now let's plot the resulting time-series with the parameter variation over-plotted for reference.

In[18]:

ake the lists numpy.arrays for easier use.

In [ ]:
RAW = numpy.array(raw_data)
TAVG = numpy.array(tavg_data)

lot raw time series

In [ ]:
figure(1)
plot(raw_time, RAW[:, 0, :, 0])
plot(t + (step-par_len)*sim.integrator.dt, a, 'r--', linewidth=2)
title("Raw -- State variable 0")

how them

In [ ]:
show()

The red dashed line is an over-plot of the stimulus on the simulated neural activity.<br>
The response of the parameter variation is visible, if you look closely, but is mostly <br>
overwhelmed by the transient from our initial conditions. If you want to see the effect <br>
of the parameter variation more clearly, simply go back and re-evaluate the cell that <br>
runs the simulation, this will continue the simulation from where it finished and <br>
once again apply the parameter variation half way through the block of simulation.<br>
Then simply evaluate the plotting cell again. Whether or not you evaluate the cell above <br>
the simulation cell again is up to you, if you do then the previously Monitored data will<br>
be cleared (data shown in the figures), if not then the newly Monitored data will be appended to the data from <br>
the previous simulation run.